In [1]:
import pandas as pd

In [60]:
bxdf[['User ID', 'ISBN', 'Book-Rating']] = bxdf[0].str.split(';', expand=True)

KeyError: 0

In [85]:
import os
import requests
import zipfile
import io

def download_and_extract_zip(zip_url, target_directory):
    try:
        # Fetch the zip data from the URL
        response = requests.get(zip_url)
        response.raise_for_status()  # Raise an exception for 4xx or 5xx errors

        # Create the target directory if it doesn't exist
        if not os.path.exists(target_directory):
            os.makedirs(target_directory)

        # Load the zip data into a ZipFile object
        with zipfile.ZipFile(io.BytesIO(response.content), 'r') as z:
            # Extract all the files from the zip archive to the target directory
            z.extractall(target_directory)

        print("Download and extraction completed successfully.")
        return True

    except requests.exceptions.RequestException as e:
        print(f"Failed to download the zip file from {zip_url}. Error: {e}")
        return False
    except zipfile.BadZipFile as e:
        print(f"Failed to extract the zip file. Error: {e}")
        return False

# Usage example:
zip_url = 'http://www2.informatik.uni-freiburg.de/~cziegler/BX/BX-CSV-Dump.zip'
target_directory = 'data/raw/'
success = download_and_extract_zip(zip_url, target_directory)

if success:
    print(f"Files have been extracted to '{target_directory}'.")

Download and extraction completed successfully.
Files have been extracted to 'data/raw/'.


In [1]:
from scripts.download_data import DownloadData

In [2]:
dd = DownloadData()

In [3]:
dd.download_goodreads_csv()

File downloaded successfully: data/raw/goodreads/books.csv
File downloaded successfully: data/raw/goodreads/ratings.csv


In [8]:
dd.download_bookcrossing_csv()


Zip download and extraction completed successfully: data/raw/bookcrossing/


In [2]:
from scripts.process_data import ProcessData
import pandas as pd
processor = ProcessData()
final_dataframe = processor.get_final_dataframe() 

DataFrame saved to data/processed/book_reference_dataframe.pkl


In [3]:
import os
import urllib
import zipfile
import time
import pickle 
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt

def map_columns_to_int(df: pd.DataFrame, save_column_name: list, mapping_dir: str = None):
    """
    Map columns of strings in a DataFrame to integers.

    Parameters:
        df (pd.DataFrame): DataFrame containing the columns to be mapped.
        column_names (list): List of column names to be mapped.
        mapping_dir (str): Directory path to save the mapping dictionaries. If None, mappings won't be saved.

    Returns:
        pd.DataFrame: DataFrame with the mapped columns.
    """
    for column_name in df.columns:
        # Create a mapping dictionary that assigns a unique integer to each unique string in the column
        unique_values = df[column_name].unique()
        mapping_dict = {value: idx for idx, value in enumerate(unique_values)}

        # Replace the strings in the column with their corresponding integer values
        df[f"{column_name}"] = df[column_name].map(mapping_dict)

        # Save the mapping dictionary to a file if mapping_dir is provided
        if column_name == save_column_name:
            mapping_file_path = f"{mapping_dir}{column_name}_mapping.pkl"
            with open(mapping_file_path, 'wb') as f:
                pickle.dump(mapping_dict, f)
            print(f"Mapping for column '{column_name}' saved to {mapping_file_path}")

    return df


In [4]:
# Call the function to map the 'Fruit' column and save the mapping
mapping_file_path = 'fruit_mapping.pkl'
df_mapped = map_columns_to_int(final_dataframe, save_column_name='isbn_gr', mapping_dir='data/processed/')

Mapping for column 'isbn_gr' saved to data/processed/isbn_gr_mapping.pkl


In [5]:
df_mapped.head()

,user_id_gr,rating_gr,books_count_gr,isbn_gr,average_rating_gr,ratings_count_gr,work_ratings_count_gr,work_text_reviews_count_gr,ratings_1_gr,ratings_2_gr,ratings_3_gr,ratings_4_gr,ratings_5_gr,Book-Rating_bx
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
17750,0,0,1,1,1,1,1,1,1,1,1,1,1,1
25293,0,1,2,2,2,2,2,2,2,2,2,2,2,2
29017,0,0,3,3,3,3,3,3,3,3,3,3,3,3
38052,0,0,4,4,4,4,4,4,4,4,4,4,4,4


In [6]:
columns_list = df_mapped.columns.tolist()
y_columns = ['rating_gr']
x_columns = [col for col in columns_list if col not in y_columns]

In [7]:
y = df_mapped.loc[:,y_columns]
X = df_mapped.loc[:,x_columns]

# Split our data into training and test sets
X_train, X_val, y_train, y_val = train_test_split(X,y,random_state=0, test_size=0.2)

In [8]:
def prep_dataloaders(X_train,y_train,X_val,y_val,batch_size):
    # Convert training and test data to TensorDatasets
    trainset = TensorDataset(torch.from_numpy(np.array(X_train)).long(), 
                            torch.from_numpy(np.array(y_train)).float())
    valset = TensorDataset(torch.from_numpy(np.array(X_val)).long(), 
                            torch.from_numpy(np.array(y_val)).float())

    # Create Dataloaders for our training and test data to allow us to iterate over minibatches 
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
    valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=False)

    return trainloader, valloader

batchsize = 64
trainloader,valloader = prep_dataloaders(X_train,y_train,X_val,y_val,batchsize)

In [17]:
X['user_id_gr']

0             0
17750         0
25293         0
29017         0
38052         0
          ...  
59977     49684
154513    49684
222792    49684
262957    49685
59978     49685
Name: user_id_gr, Length: 264108, dtype: int64

In [14]:
np.array(X['user_id_gr'])[:,0]

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [20]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class NNHybridFiltering(nn.Module):
    def __init__(self, n_users, n_isbn, n_bookscount, n_avgratings, n_ratings, 
                 n_workratings, n_textratings, n_ratings1, n_ratings2, n_ratings3,
                 n_ratings4, n_ratings5, n_bxrating, embdim_users, embdim_isbn, 
                 embdim_bookscount, embdim_avgratings, embdim_ratings, embdim_workratings,
                 embdim_textratings, embdim_ratings1, embdim_ratings2, embdim_ratings3,
                 embdim_ratings4, embdim_ratings5, embdim_bxrating, n_activations,
                 rating_range):

        super().__init__()

        self.user_embeddings = nn.Embedding(num_embeddings=n_users, embedding_dim=embdim_users)
        self.isbn_embeddings = nn.Embedding(num_embeddings=n_isbn, embedding_dim=embdim_isbn)
        self.bookscount_embeddings = nn.Embedding(num_embeddings=n_bookscount, embedding_dim=embdim_bookscount)
        self.avgratings_embeddings = nn.Embedding(num_embeddings=n_avgratings, embedding_dim=embdim_avgratings)
        self.ratings_embeddings = nn.Embedding(num_embeddings=n_ratings, embedding_dim=embdim_ratings)
        self.workratings_embeddings = nn.Embedding(num_embeddings=n_workratings, embedding_dim=embdim_workratings)
        self.textratings_embeddings = nn.Embedding(num_embeddings=n_textratings, embedding_dim=embdim_textratings)
        self.ratings1_embeddings = nn.Embedding(num_embeddings=n_ratings1, embedding_dim=embdim_ratings1)
        self.ratings2_embeddings = nn.Embedding(num_embeddings=n_ratings2, embedding_dim=embdim_ratings2)
        self.ratings3_embeddings = nn.Embedding(num_embeddings=n_ratings3, embedding_dim=embdim_ratings3)
        self.ratings4_embeddings = nn.Embedding(num_embeddings=n_ratings4, embedding_dim=embdim_ratings4)
        self.ratings5_embeddings = nn.Embedding(num_embeddings=n_ratings5, embedding_dim=embdim_ratings5)
        self.bxrating_embeddings = nn.Embedding(num_embeddings=n_bxrating, embedding_dim=embdim_bxrating)

        embdim_sum = (
            embdim_users + embdim_isbn + embdim_bookscount + embdim_avgratings +
            embdim_ratings + embdim_workratings + embdim_textratings +
            embdim_ratings1 + embdim_ratings2 + embdim_ratings3 +
            embdim_ratings4 + embdim_ratings5 + embdim_bxrating
        )

        self.fc1 = nn.Linear(embdim_sum, n_activations)
        self.fc2 = nn.Linear(n_activations, 1)
        self.rating_range = rating_range

    def forward(self, X):
        # Get embeddings for minibatch
        embedded_users = self.user_embeddings(X['user_id_gr'])
        embedded_isbn = self.isbn_embeddings(X['isbn_gr'])
        embedded_bookscount = self.bookscount_embeddings(X['books_count_gr'])
        embedded_avgratings = self.avgratings_embeddings(X['average_rating_gr'])
        embedded_ratings = self.ratings_embeddingsy(X['ratings_count_gr'])
        embedded_workratings = self.workratings_embeddings(X['work_ratings_count_gr'])
        embedded_textratings = self.textratings_embeddings(X['work_text_reviews_count_gr'] )
        embedded_ratings1 = self.ratings1_embeddings(X['ratings_1_gr'])
        embedded_ratings2 = self.ratings1_embeddings(X['ratings_2_gr'])
        embedded_ratings3 = self.ratings3_embeddings(X['ratings_3_gr'])
        embedded_ratings4 = self.ratings4_embeddings(X['ratings_4_gr'])
        embedded_ratings5 = self.ratings5_embeddings(X['ratings_5_gr'])
        embedded_bxrating = self.bxrating_embeddings(X['Book-Rating_bx'])

        # Concatenate user, item and genre embeddings
        embeddings = torch.cat([embedded_users, embedded_isbn, embedded_bookscount, 
                                embedded_avgratings, embedded_ratings, embedded_workratings,
                                embedded_textratings, embedded_ratings1, embedded_ratings2,
                                embedded_ratings3, embedded_ratings4, embedded_ratings5, embedded_bxrating
                                ], dim=1)

        # Pass embeddings through network
        preds = self.fc1(embeddings)
        preds = F.relu(preds)
        preds = self.fc2(preds)
        # Scale predicted ratings to target-range [low, high]
        preds = torch.sigmoid(preds) * (self.rating_range[1] - self.rating_range[0]) + self.rating_range[0]
        return preds


In [21]:
np.array(X['user_id_gr']).reshape(-1,1)[:,0]

array([    0,     0,     0, ..., 49684, 49685, 49685], dtype=int64)

In [22]:
def train_model(model, criterion, optimizer, dataloaders, device, num_epochs=5, scheduler=None):
    model = model.to(device) # Send model to GPU if available
    since = time.time()

    costpaths = {'train':[],'val':[]}

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0

            # Get the inputs and labels, and send to GPU if available
            for (inputs,labels) in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Zero the weight gradients
                optimizer.zero_grad()

                # Forward pass to get outputs and calculate loss
                # Track gradient only for training data
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model.forward(inputs).view(-1)
                    loss = criterion(outputs, labels)

                    # Backpropagation to get the gradients with respect to each weight
                    # Only if in train
                    if phase == 'train':
                        loss.backward()
                        # Update the weights
                        optimizer.step()

                # Convert loss into a scalar and add it to running_loss
                running_loss += np.sqrt(loss.item()) * labels.size(0)

            # Step along learning rate scheduler when in train
            if (phase == 'train') and (scheduler is not None):
                scheduler.step()

            # Calculate and display average loss and accuracy for the epoch
            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            costpaths[phase].append(epoch_loss)
            print('{} loss: {:.4f}'.format(phase, epoch_loss))

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))

    return costpaths


In [23]:
# Train the model
dataloaders = {'train':trainloader, 'val':valloader}
n_users = X.loc[:,'user_id_gr'].max() + 1
n_isbn = X.loc[:,'isbn_gr'].max() + 1
n_bookscount = X.loc[:,'books_count_gr'].max() + 1
n_avgratings = X.loc[:,'average_rating_gr'].max() + 1
n_ratings = X.loc[:,'ratings_count_gr'].max() + 1
n_workratings = X.loc[:,'work_ratings_count_gr'].max() + 1
n_textratings = X.loc[:,'work_text_reviews_count_gr'].max() + 1
n_ratings1 = X.loc[:,'ratings_1_gr'].max() + 1
n_ratings2 = X.loc[:,'ratings_2_gr'].max() + 1
n_ratings3 = X.loc[:,'ratings_3_gr'].max() + 1
n_ratings4 = X.loc[:,'ratings_4_gr'].max() + 1
n_ratings5 = X.loc[:,'ratings_5_gr'].max() + 1
n_bxrating = X.loc[:,'Book-Rating_bx'].max() + 1

embdim_users = 50
embdim_isbn = 50
embdim_bookscount = 50
embdim_avgratings = 50
embdim_ratings = 50
embdim_workratings = 50
embdim_textratings = 50
embdim_ratings1 = 50
embdim_ratings2 = 50
embdim_ratings3 = 50
embdim_ratings4 = 50
embdim_ratings5 = 50
embdim_bxrating = 50

model = NNHybridFiltering(                
                        n_users = n_users,
                        n_isbn = n_isbn, 
                        n_bookscount = n_bookscount,
                        n_avgratings = n_avgratings,
                        n_ratings = n_ratings, 
                        n_workratings = n_workratings, 
                        n_textratings = n_textratings,
                        n_ratings1 = n_ratings1, 
                        n_ratings2 = n_ratings2, 
                        n_ratings3 = n_ratings3, 
                        n_ratings4 = n_ratings4, 
                        n_ratings5 = n_ratings5, 
                        n_bxrating = n_bxrating,
                        embdim_users = embdim_users,
                        embdim_isbn = embdim_isbn, 
                        embdim_bookscount = embdim_bookscount,
                        embdim_avgratings = embdim_avgratings, 
                        embdim_ratings = embdim_ratings,
                        embdim_workratings = embdim_workratings,
                        embdim_textratings = embdim_textratings,
                        embdim_ratings1 = embdim_ratings1,
                        embdim_ratings2 = embdim_ratings2, 
                        embdim_ratings3 = embdim_ratings3, 
                        embdim_ratings4 = embdim_ratings4, 
                        embdim_ratings5 = embdim_ratings5, 
                        embdim_bxrating = embdim_bxrating,
                        n_activations= 100,
                        rating_range=[0.,4.])
                       
criterion = nn.MSELoss()
lr=0.001
n_epochs=10
wd=1e-3
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

cost_paths = train_model(model,criterion,optimizer,dataloaders, device,n_epochs, scheduler=None)

Epoch 0/9
----------


IndexError: too many indices for tensor of dimension 2

In [27]:
X['user_id_gr']

0             0
17750         0
25293         0
29017         0
38052         0
          ...  
59977     49684
154513    49684
222792    49684
262957    49685
59978     49685
Name: user_id_gr, Length: 264108, dtype: int64